
---

### Reminder: This 📘 `.NET Interactive` notebook needs to be run from VS Code with [these prerequisites](../PREREQS.md).

#### How to use this notebook: 

* Just read the text and scroll along until you run into code blocks.
* Code blocks have computer code inside them — hover over the block and you can run the code.
* Run the code by hitting the ▶️ "play" button to the left. If the code runs you'll see a ✔️. If not, you'll get a ❌.
* The output and status of the code block will appear just below itself — you need to scroll down further to see it.
* Sometimes a code block will ask you for input in a hard-to-notice dialog box 👆 at the top of your notebook window. 

---

# Recipe V: 🍋 Connectors w/ Bing
## 🧑‍🍳 Cook using fresher context with live data from APIs

## Step 1: Instantiate a 🔥 kernel 

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.9.61.1-preview"
#r "nuget: Microsoft.SemanticKernel.Skills.Web, 0.9.61.1-preview"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;
using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.Orchestration;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernel.Config.AddAzureOpenAITextCompletion("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletion("davinci", model, apiKey, orgId);

## Step 2: Integrate live data with the Bing Web Search API 🍋 connector

To get a Bing Web Search API key visit [this resource](https://www.microsoft.com/en-us/bing/apis/bing-web-search-api) to start. Once you have that in hand, you can now grab live search information to get added to the context of your choice. For example, we can find out what's the tallest building in New York City:

In [ ]:
using Microsoft.SemanticKernel.Skills.Web;
using Microsoft.SemanticKernel.Skills.Web.Bing;

var bingApiKey = await InteractiveKernel.GetPasswordAsync("Please enter your Bing Search API key");
var bingConnector = new BingConnector(bingApiKey);

var webSearchEngineSkill = new WebSearchEngineSkill(bingConnector);
var web = kernel.ImportSkill(webSearchEngineSkill);

var bingResult = await kernel.RunAsync(
    "What's the tallest building in NYC?", 
    web["SearchAsync"]
);
Console.WriteLine(bingResult);

> ✅ Keep in mind that getting a Bing Web Search API key will require you to have a fresh FREE Azure account, or else a paid Azure account with active billing credentials.

> 🤔 **Get `"Error: Response status code does not indicate success: 401 (PermissionDenied)."` message?** This could happen if you just provisioned a Bing Web Search API. Wait a few minutes and try again. 

> 😱 **Get a different error message?** go to https://aka.ms/sk/discord where we have realtime support available to troubleshoot your problem.

## Step 3: Combine existing 🥑 memories with fresh 🍋 data

So we take our memories to the next level by providing it "live" memories that are delivered via a connector. It's a terrific combination of both _timeless_ information and _timely_ information. We feed the output of the `SearchAsync` function into a custom-crafted inline semantic function to tell a short story while including a few "facts" as inline 🥑 memories, like:

1. I'm tall-ish: about 5ft 11in
2. I like sushi — but not the kind with uni

And we'll add a fresh memory that we grab from a Bing 🍋 connector to brag that we've been to `the tallest building in NYC` by adding a 3rd fact of:

3. I've been to <the tallest building in NYC>

In [ ]:
var myInlineSemanticFunction = kernel.CreateSemanticFunction(@"
These are facts about me:
1. I'm tall-ish: about 5ft 11in
2. I like sushi — but not the kind with uni
3. I've been to {{$input}}
Here is a short story under 100 words about me:
", maxTokens: 200, temperature: 0.1, topP: .1);

var result = await kernel.RunAsync(
    "What's the tallest building in NYC?", 
    web["SearchAsync"],
    myInlineSemanticFunction
);

Console.WriteLine(result);

Now you can see how it's possible to use Semantic Kernel in a variety of ways — from designing prompts, expanding a prompt's context with memories, and now drawing live data into an LLM AI's processing capability.

## Step 4: Let's go one step further and add some extra seasoning with the `SummarizeSkill.Notegen` 🧂 function

In [ ]:
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");
var mySkill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "SummarizeSkill");

var result = await kernel.RunAsync(
    "What's the tallest building in NYC?", 
    web["SearchAsync"],
    myInlineSemanticFunction,
    mySkill["Notegen"]
);

// Return the result to the Notebook
Console.WriteLine(result);

If you look closely at the code, what we did is make a "chain" of functions. We'll be doing that in our next recipe section in the context of a well-known process for innovation called "Design Thinking."

# ⏭️ Next Steps

Run through more advanced examples in the notebooks that are available in our GitHub repo at [https://aka.ms/sk/repo](https://aka.ms/sk/repo).

[Design thinking with 🔥 kernel is up next!](../e6-design-chain/notebook.ipynb)

Or stay a longer while and ask Bing to added new information from the Internet to fortify your LLM AI magic. And you can also try different functions within the `SummarizeSkill` to see different outputs in Step 4.